In [1]:
!pip install regex

     |████████████████████████████████| 733 kB 15.3 MB/s eta 0:00:01


In [2]:
import regex as re
import operator

In [3]:
# Treshold coverage for calling
cov_threshold = 2

# Variant allele frequency coverage for calling
vaf_threshold = 0.2

# Threshlod to call the variant heterozygous (GT = 0/1) which means that only one chromosome has found variant
# We set here same threshold as for variant allele frequency
het_threshold = vaf_threshold

# Threshlod to call the variant homozygous (GT = 1/1) which means that both chromosomes have found variant
hom_threshold = 0.8

# Threshold for quality of bases. If mean base quality is below this threshold we set filter tag for this variant
qual_threshold = 40

# These two types of mutations (deletions and insertions) are commented
# but we gave an regex example how these should be parsed

#deletion_reg = re.compile("-[0-9]+[ACGTNacgtn]+")
#insertion_reg = re.compile("\+[0-9]+[ACGTNacgtn]+")

In [4]:
def alternative_base(reads):
    
    """Function for counting alternative bases from reads part from pileup file.
    More about pileup which is created from BAM file can be found here: 
    https://en.wikipedia.org/wiki/Pileup_format
    Pileup is created out of BAM file. 
    Example task where pileup file is created can be found here: 
    https://cgc.sbgenomics.com/u/ana_djukic/matf-2021/tasks/d8563fba-1576-4755-bdb5-e26251dcdab0/"""
    
    alt_dict = {} # create dict structure where keys will be variants and values number of each variant found on specific position
    alt_base = "" # variable for storing info about variant found in pileup
    alt_alleles = 0 # variable for storing info how many variants are found in pileup
    
    # For example pileup reads are: chr1 T 22 ..^c..,,.,.,.C.,,,.,..c. 
    # This means that we have two variants, since characater after sign "^" gives mapping quality not the alternative bases
    alt_base_reg = re.compile("(?<!\^)[ACTGNactgn]") 
    alt_bases_list = [x.upper() for x in alt_base_reg.findall(reads)]  # upper all string bases in pileup 
    
    # Check if there is mutation and if "N" isn't only one in list
    if alt_bases_list != [] and "N" not in set(alt_bases_list):
        if "A" in alt_bases_list:
            # Count number of mutations "A"
            alt_dict["A"] = alt_bases_list.count("A")
        elif "C" in alt_bases_list:
            # Count number of mutations "C"
            alt_dict["C"] = alt_bases_list.count("C")
        elif "G" in alt_bases_list:
            # Count number of mutations "G"
            alt_dict["G"] = alt_bases_list.count("G")
        elif "T" in alt_bases_list:
            # Count number of mutations "T"
            alt_dict["T"] = alt_bases_list.count("T")
        
        # Take the maximum number of alternative bases as real alternative base.
        # Note: Here, we only took maximum, but different algorithms use different statistics to calculate this
        alt_base = max(alt_dict, key = alt_dict.get)
        # When calculated just take alt base from the created dict
        alt_alleles = alt_dict[alt_base]
        
    return alt_base, alt_alleles

def calculate_vaf(num_alt_bases, coverage):
    """Calculation of Variant allele frequency from
    number of alternative bases and coverage."""
    # Example, pileup reads are: chr1 T 22 ..^c..,,.,.,.C.,,,.,..c. 
    # Here we have 22 reads aligned on that position, and 2 alternative bases, which gives us VAF: 2/22
    vaf = round((num_alt_bases/coverage),2)
    
    return vaf

def genotype(vaf, het_threshold, hom_threshold):
    """Genotype variants based on thresholds we set as global variables"""
    if (vaf > hom_threshold):
        GT = "1/1"
    elif (vaf < hom_threshold and vaf > het_threshold):
        GT = "0/1"
    else:
        GT = "0/0"
        
    return GT
    
def mean_quality(reads, quals):
    """Calculate mean base quality of variant"""
    # ?<!\^ Negative lookbehind for ^ char and letters after it to calculate average base quality of found bases
    alt_base_reg = re.compile("(?<!\^)[ACTGNactgn]")
    altbasesIter = alt_base_reg.finditer(reads)
    # Take indexes of bases quality which are found as mutations 
    alt_indexes = [alt_ind.start(0) for alt_ind in altbasesIter]
    # Create list of qualities
    qual_list = [qual for qual in quals]
    # Phred Quality of those bases found in qual list, represented in ASCII with -33 offset
    qualities = [(ord(qual_list[i]) - 33) for i in alt_indexes]
    # Calculate mean quality of Phred qualities extracted
    mean_qual = str(round(sum(qualities)/len(qualities),2))
    
    return mean_qual

def split_line(line):
    """Split line by tab to important columns and extract important columns.
    Indexes are taken as default with known pileup format."""
    parts = line.strip('\n').split('\t')
    chrom = parts[0]
    pos = parts[1]
    ref_base = parts[2]
    cov = int(parts[3])
    reads = parts[-2].replace("$", "")
    qual_part = parts[-1]
    
    return chrom, pos, ref_base, cov, reads, qual_part
    
def quality_filter(mean_qual):
    """Filter variants by quality"""
    mean_qual = float(mean_qual)
    if mean_qual > qual_threshold:
        filt = "PASS"
    else:
        filt = "LowQual"
        
    return filt

In [5]:
# Open mpileup for reading and VCF for writing
# example task with pileup output file can be found here: 
# https://cgc.sbgenomics.com/u/ana_djukic/matf-2021/tasks/d8563fba-1576-4755-bdb5-e26251dcdab0/

with open('/sbgenomics/project-files/ < FILE NAME >', 'r') as pileup, open('test-merged-normal.vcf', 'w') as vcf:
    
    # Add header of VCF by its specification
    vcf.write("##fileformat=VCFv4.2\n")
    
    # Add header columns in VCF
    vcf.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tSAMPLE\n")
    
    for line in pileup:
        
        # We split and return parts of each pileup line
        chrom, pos, ref_base, cov, reads, qual_part = split_line(line)
        
        # If coverage larger than threshold then parse reads line
        if cov > cov_threshold:
            # Retrun alternative base and number of alternative alleles
            alt, alt_allels = alternative_base(reads) 
            
            # Count Variant allele frequency
            vaf = calculate_vaf(alt_allels, cov)
            
            # If VAF is larger than threshold 
            # and length of bases in pileup is equal of length of qualities
            if vaf > vaf_threshold and (len(reads) == len(qual_part)):
                
                # Return mean quality
                mean_qual = mean_quality(reads,qual_part)
                
                # Return filter field
                filt = quality_filter(mean_qual)
                
                # Return calculated GT 
                GT = genotype(vaf, het_threshold, hom_threshold)
                
                # Create sample column
                sample_string = ":".join([GT, str(vaf),str(cov), str(alt_allels)])
                
                # Create line which will be written in VCF
                line_to_write = '\t'.join([chrom, pos, '.', ref_base, alt, 
                                           mean_qual, filt, '.', "GT:VAF:DP:AD", 
                                           sample_string])
                # Write line to VCF    
                vcf.write(line_to_write + '\n')
        
        # If coverage is lower than threshold skip this position            
        else:
            continue